In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# === Step 1: Load and Prepare the Dataset ===

# Define the file path
file_path = 'Opportunity_Set.xlsx'

# Read the Excel file, parse 'Date' as datetime, and set it as the index
try:
    data = pd.read_excel(file_path, parse_dates=['Date'], index_col='Date')
except FileNotFoundError:
    raise FileNotFoundError(f"The file '{file_path}' was not found.")
except ValueError as e:
    raise ValueError(f"Error parsing the Excel file: {e}")

# Verify that 'Vanguard LifeStrategy Income Fund (VASIX)' exists in the dataset
vasix_col = 'Vanguard LifeStrategy Income Fund (VASIX)'
if vasix_col not in data.columns:
    raise ValueError(f"Column '{vasix_col}' not found in the dataset.")

# Exclude the benchmark and ensure only numerical asset columns are retained
asset_columns = [col for col in data.columns if col not in [vasix_col]]
assets = data[asset_columns]

# Convert all asset columns to numeric, coercing errors to NaN, and drop rows with any NaN values
assets = assets.apply(pd.to_numeric, errors='coerce').dropna()

# Verify that there are enough data points for analysis
if assets.shape[0] < 2:
    raise ValueError("Insufficient data after cleaning. At least two data points are required.")

# === Step 2: Calculate Covariance Matrix ===

cov_matrix = assets.cov()

# === Step 3: Define Portfolio Optimization Functions ===

def portfolio_volatility(weights, cov_matrix):
    """Calculate the portfolio volatility."""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def risk_contribution(weights, cov_matrix):
    """Calculate the risk contribution of each asset to the portfolio."""
    total_vol = portfolio_volatility(weights, cov_matrix)
    marginal_contrib = np.dot(cov_matrix, weights)
    return (marginal_contrib * weights) / total_vol

def risk_parity_objective(weights, cov_matrix):
    """Objective function to minimize the standard deviation of risk contributions."""
    contribs = risk_contribution(weights, cov_matrix)
    return np.std(contribs)

# === Step 4: Set Up Optimization Constraints and Bounds ===

# Constraint: The sum of weights must equal 1
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

# Bounds: Weights must be between 0 and 1
bounds = [(0, 1) for _ in range(len(asset_columns))]

# Initial guess: Equally weighted portfolio
init_guess = np.ones(len(asset_columns)) / len(asset_columns)

# === Step 5: Perform the Optimization ===

result = minimize(
    fun=risk_parity_objective,
    x0=init_guess,
    args=(cov_matrix,),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Check if the optimization was successful
if not result.success:
    raise ValueError(f"Optimization failed: {result.message}")

# Extract the optimized weights
optimal_weights = result.x

# === Step 6: Calculate Risk Contributions and Portfolio Standard Deviation ===

# Calculate the portfolio's standard deviation
portfolio_std = portfolio_volatility(optimal_weights, cov_matrix)

# Calculate Risk Contributions
risk_contribs = risk_contribution(optimal_weights, cov_matrix)

# Create a DataFrame to display weights, risk contributions, and risk contribution percentages
risk_contrib_table = pd.DataFrame({
    'Asset': asset_columns,
    'Weight %': optimal_weights * 100,
    'Risk Contribution': risk_contribs,
    'Risk Contribution %': risk_contribs / portfolio_std * 100  # Convert to percentage
})

# Convert the weights to percentages and round to two decimal places
risk_contrib_table['Weight %'] = risk_contrib_table['Weight %'].round(2)
risk_contrib_table['Risk Contribution %'] = risk_contrib_table['Risk Contribution %'].round(2)

# Display the Risk Contribution Table with updated weights and contributions
print(f"Portfolio Standard Deviation (monthly): {portfolio_std:.6f}\n")
print("Risk Contribution to Portfolio Standard Deviation:")
print(risk_contrib_table.to_string(index=False))

# === Step 7: Calculate Portfolio and Benchmark Returns ===

# Calculate portfolio returns using the optimized weights
portfolio_returns = assets.dot(optimal_weights)

# Extract benchmark returns
benchmark_returns = data[vasix_col].loc[portfolio_returns.index]

# Combine portfolio and benchmark returns into a single DataFrame
performance_df = pd.DataFrame({
    'Risk Parity Portfolio': portfolio_returns,
    'VASIX (Benchmark)': benchmark_returns
}).dropna()

# Ensure the index is datetime and sorted
performance_df.index = pd.to_datetime(performance_df.index)
performance_df = performance_df.sort_index()

# === Step 8: Define Performance Metrics Functions ===

def calculate_cagr(df, column_name):
    """Calculate the Compound Annual Growth Rate (CAGR)."""
    if df.index.nunique() < 2:
        raise ValueError(f"Date range is too short to calculate CAGR for {column_name}.")
    
    # Calculate the number of years between the first and last date
    days_diff = (df.index[-1] - df.index[0]).days
    if days_diff <= 0:
        raise ValueError("Date range is too short to calculate CAGR.")
    
    years = days_diff / 365.25
    cumulative_return = (1 + df[column_name]).prod()
    
    return (cumulative_return ** (1 / years) - 1) * 100  # Convert to percentage

def calculate_max_drawdown(df, column_name):
    """Calculate the Maximum Drawdown."""
    cumulative_return = (1 + df[column_name]).cumprod()
    rolling_max = cumulative_return.cummax()
    drawdown = (cumulative_return - rolling_max) / rolling_max
    return drawdown.min() * 100  # Convert to percentage

# === Step 9: Calculate Annual Returns ===

def calculate_annual_returns(df, column_name):
    """Calculate annual returns for the portfolio."""
    # Resample data to year-end using 'YE' for year-end
    annual_returns = df[column_name].resample('YE').apply(lambda x: (1 + x).prod() - 1)
    
    # Convert to percentage
    return (annual_returns * 100).round(2)

# Calculate annual returns for both portfolio and benchmark
annual_returns_portfolio = calculate_annual_returns(performance_df, 'Risk Parity Portfolio')
annual_returns_benchmark = calculate_annual_returns(performance_df, 'VASIX (Benchmark)')

# Combine annual returns into a DataFrame
annual_returns_table = pd.DataFrame({
    'Risk Parity Portfolio (%)': annual_returns_portfolio,
    'VASIX (Benchmark) (%)': annual_returns_benchmark
})

# Display the annual return table
print("\nAnnual Return Table:")
print(annual_returns_table.to_string())


# === Step 10: Calculate Performance Metrics ===

if performance_df.index.nunique() < 2:
    print("Date range is too short for CAGR calculation. Skipping CAGR.")
else:
    try:
        # Calculate CAGR for both portfolio and benchmark
        cagr_portfolio = calculate_cagr(performance_df, 'Risk Parity Portfolio')
        cagr_benchmark = calculate_cagr(performance_df, 'VASIX (Benchmark)')
        
        # Calculate Standard Deviation (annualized and converted to percentage)
        std_portfolio = portfolio_returns.std() * np.sqrt(12) * 100  # Assuming monthly returns
        std_benchmark = benchmark_returns.std() * np.sqrt(12) * 100
        
        # Calculate Maximum Drawdown for both portfolio and benchmark
        mdd_portfolio = calculate_max_drawdown(performance_df, 'Risk Parity Portfolio')
        mdd_benchmark = calculate_max_drawdown(performance_df, 'VASIX (Benchmark)')
        
        # Compile the performance metrics into a DataFrame
        performance_metrics = pd.DataFrame({
            'CAGR (%)': [cagr_portfolio, cagr_benchmark],
            'Standard Deviation (%)': [std_portfolio, std_benchmark],
            'Maximum Drawdown (%)': [mdd_portfolio, mdd_benchmark]
        }, index=['Risk Parity Portfolio', 'VASIX (Benchmark)'])
        
        # Format and display the Performance Metrics with two decimal places
        print("\nPerformance Metrics (in percentages):")
        print(performance_metrics.round(2).to_string())
        
    except ValueError as e:
        print(f"Error calculating performance metrics: {e}")


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# === Step 1: Load and Prepare the Dataset ===

# Define the file path
file_path = 'Opportunity_Set.xlsx'

# Read the Excel file, parse 'Date' as datetime, and set it as the index
try:
    data = pd.read_excel(file_path, parse_dates=['Date'], index_col='Date')
except FileNotFoundError:
    raise FileNotFoundError(f"The file '{file_path}' was not found.")
except ValueError as e:
    raise ValueError(f"Error parsing the Excel file: {e}")

# Verify that 'Vanguard LifeStrategy Income Fund (VASIX)' exists in the dataset
vasix_col = 'Vanguard LifeStrategy Income Fund (VASIX)'
if vasix_col not in data.columns:
    raise ValueError(f"Column '{vasix_col}' not found in the dataset.")

# Exclude the benchmark and ensure only numerical asset columns are retained
asset_columns = [col for col in data.columns if col not in [vasix_col]]
assets = data[asset_columns]

# Convert all asset columns to numeric, coercing errors to NaN, and drop rows with any NaN values
assets = assets.apply(pd.to_numeric, errors='coerce').dropna()

# Verify that there are enough data points for analysis
if assets.shape[0] < 2:
    raise ValueError("Insufficient data after cleaning. At least two data points are required.")

# === Step 2: Calculate Covariance Matrix ===

cov_matrix = assets.cov()

# === Step 3: Define Portfolio Optimization Functions ===

def portfolio_volatility(weights, cov_matrix):
    """Calculate the portfolio volatility."""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def risk_contribution(weights, cov_matrix):
    """Calculate the risk contribution of each asset to the portfolio."""
    total_vol = portfolio_volatility(weights, cov_matrix)
    marginal_contrib = np.dot(cov_matrix, weights)
    return (marginal_contrib * weights) / total_vol

# === Step 4: Minimum Variance Portfolio Objective Function ===

def min_variance_objective(weights, cov_matrix):
    """Objective function to minimize portfolio variance."""
    return portfolio_volatility(weights, cov_matrix)

# === Step 5: Set Up Optimization Constraints and Bounds ===

# Constraint: The sum of weights must equal 1
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

# Bounds: Weights must be between 0 and 1 (long-only constraint)
bounds = [(0, 1) for _ in range(len(asset_columns))]

# Initial guess: Equally weighted portfolio
init_guess = np.ones(len(asset_columns)) / len(asset_columns)

# === Step 6: Perform the Optimization for Minimum Variance Portfolio ===

result = minimize(
    fun=min_variance_objective,
    x0=init_guess,
    args=(cov_matrix,),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Check if the optimization was successful
if not result.success:
    raise ValueError(f"Optimization failed: {result.message}")

# Extract the optimized weights for the minimum variance portfolio
optimal_weights = result.x

# === Step 7: Calculate Risk Contributions and Portfolio Standard Deviation ===

# Calculate the portfolio's standard deviation
portfolio_std = portfolio_volatility(optimal_weights, cov_matrix)

# Calculate Risk Contributions
risk_contribs = risk_contribution(optimal_weights, cov_matrix)

# Create a DataFrame to display weights, risk contributions, and risk contribution percentages
risk_contrib_table = pd.DataFrame({
    'Asset': asset_columns,
    'Weight %': optimal_weights * 100,
    'Risk Contribution': risk_contribs,
    'Risk Contribution %': risk_contribs / portfolio_std * 100  # Convert to percentage
})

# Convert the weights to percentages and round to two decimal places
risk_contrib_table['Weight %'] = risk_contrib_table['Weight %'].round(2)
risk_contrib_table['Risk Contribution %'] = risk_contrib_table['Risk Contribution %'].round(2)

# Display the Risk Contribution Table with updated weights and contributions
print(f"Portfolio Standard Deviation (monthly): {portfolio_std:.6f}\n")
print("Risk Contribution to Portfolio Standard Deviation:")
print(risk_contrib_table.to_string(index=False))

# === Step 8: Calculate Portfolio Returns ===

# Calculate portfolio returns using the optimized weights
portfolio_returns = assets.dot(optimal_weights)

# Extract benchmark returns
benchmark_returns = data[vasix_col].loc[portfolio_returns.index]

# Combine portfolio and benchmark returns into a single DataFrame
performance_df = pd.DataFrame({
    'Minimum Variance Portfolio': portfolio_returns,
    'VASIX (Benchmark)': benchmark_returns
}).dropna()

# Ensure the index is datetime and sorted
performance_df.index = pd.to_datetime(performance_df.index)
performance_df = performance_df.sort_index()

# === Step 9: Define Performance Metrics Functions ===

def calculate_cagr(df, column_name):
    """Calculate the Compound Annual Growth Rate (CAGR)."""
    if df.index.nunique() < 2:
        raise ValueError(f"Date range is too short to calculate CAGR for {column_name}.")
    
    # Calculate the number of years between the first and last date
    days_diff = (df.index[-1] - df.index[0]).days
    if days_diff <= 0:
        raise ValueError("Date range is too short to calculate CAGR.")
    
    years = days_diff / 365.25
    cumulative_return = (1 + df[column_name]).prod()
    
    return (cumulative_return ** (1 / years) - 1) * 100  # Convert to percentage

def calculate_max_drawdown(df, column_name):
    """Calculate the Maximum Drawdown."""
    cumulative_return = (1 + df[column_name]).cumprod()
    rolling_max = cumulative_return.cummax()
    drawdown = (cumulative_return - rolling_max) / rolling_max
    return drawdown.min() * 100  # Convert to percentage

# === Step 10: Calculate Annual Returns ===

def calculate_annual_returns(df, column_name):
    """Calculate annual returns for the portfolio."""
    # Resample data to year-end using 'YE' for year-end
    annual_returns = df[column_name].resample('YE').apply(lambda x: (1 + x).prod() - 1)
    
    # Convert to percentage
    return (annual_returns * 100).round(2)

# Calculate annual returns for both portfolio and benchmark
annual_returns_portfolio = calculate_annual_returns(performance_df, 'Minimum Variance Portfolio')
annual_returns_benchmark = calculate_annual_returns(performance_df, 'VASIX (Benchmark)')

# Combine annual returns into a DataFrame
annual_returns_table = pd.DataFrame({
    'Minimum Variance Portfolio (%)': annual_returns_portfolio,
    'VASIX (Benchmark) (%)': annual_returns_benchmark
})

# Display the annual return table
print("\nAnnual Return Table:")
print(annual_returns_table.to_string())

# === Step 11: Calculate Performance Metrics ===

if performance_df.index.nunique() < 2:
    print("Date range is too short for CAGR calculation. Skipping CAGR.")
else:
    try:
        # Calculate CAGR for both portfolio and benchmark
        cagr_portfolio = calculate_cagr(performance_df, 'Minimum Variance Portfolio')
        cagr_benchmark = calculate_cagr(performance_df, 'VASIX (Benchmark)')
        
        # Calculate Standard Deviation (annualized and converted to percentage)
        std_portfolio = portfolio_returns.std() * np.sqrt(12) * 100  # Assuming monthly returns
        std_benchmark = benchmark_returns.std() * np.sqrt(12) * 100
        
        # Calculate Maximum Drawdown for both portfolio and benchmark
        mdd_portfolio = calculate_max_drawdown(performance_df, 'Minimum Variance Portfolio')
        mdd_benchmark = calculate_max_drawdown(performance_df, 'VASIX (Benchmark)')
        
        # Compile the performance metrics into a DataFrame
        performance_metrics = pd.DataFrame({
            'CAGR (%)': [cagr_portfolio, cagr_benchmark],
            'Standard Deviation (%)': [std_portfolio, std_benchmark],
            'Maximum Drawdown (%)': [mdd_portfolio, mdd_benchmark]
        }, index=['Minimum Variance Portfolio', 'VASIX (Benchmark)'])
        
        # Format and display the Performance Metrics with two decimal places
        print("\nPerformance Metrics (in percentages):")
        print(performance_metrics.round(2).to_string())
        
    except ValueError as e:
        print(f"Error calculating performance metrics: {e}")


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# === Step 1: Load and Prepare the Dataset ===

# Define the file path
file_path = 'Opportunity_Set.xlsx'

# Read the Excel file, parse 'Date' as datetime, and set it as the index
try:
    data = pd.read_excel(file_path, parse_dates=['Date'], index_col='Date')
except FileNotFoundError:
    raise FileNotFoundError(f"The file '{file_path}' was not found.")
except ValueError as e:
    raise ValueError(f"Error parsing the Excel file: {e}")

# Verify that 'Vanguard LifeStrategy Income Fund (VASIX)' exists in the dataset
vasix_col = 'Vanguard LifeStrategy Income Fund (VASIX)'
if vasix_col not in data.columns:
    raise ValueError(f"Column '{vasix_col}' not found in the dataset.")

# Exclude the benchmark and ensure only numerical asset columns are retained
asset_columns = [col for col in data.columns if col not in [vasix_col]]
assets = data[asset_columns]

# Convert all asset columns to numeric, coercing errors to NaN, and drop rows with any NaN values
assets = assets.apply(pd.to_numeric, errors='coerce').dropna()

# Verify that there are enough data points for analysis
if assets.shape[0] < 2:
    raise ValueError("Insufficient data after cleaning. At least two data points are required.")

# === Step 2: Calculate Covariance Matrix and Standard Deviations ===

cov_matrix = assets.cov()
asset_std = assets.std()  # Standard deviation (volatility) of individual assets

# === Step 3: Define Maximum Diversification Objective Function ===

def portfolio_volatility(weights, cov_matrix):
    """Calculate the portfolio volatility."""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def diversification_ratio(weights, asset_std, cov_matrix):
    """Calculate the Diversification Ratio."""
    weighted_volatility_sum = np.dot(weights, asset_std)  # Sum of weighted asset volatilities
    portfolio_vol = portfolio_volatility(weights, cov_matrix)  # Portfolio volatility
    return weighted_volatility_sum / portfolio_vol

def max_diversification_objective(weights, asset_std, cov_matrix):
    """Objective function to maximize the diversification ratio."""
    return -diversification_ratio(weights, asset_std, cov_matrix)  # Minimize the negative of DR

# === Step 4: Set Up Optimization Constraints and Bounds ===

# Constraint: The sum of weights must equal 1
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

# Bounds: Weights must be between 0 and 1 (long-only constraint)
bounds = [(0, 1) for _ in range(len(asset_columns))]

# Initial guess: Equally weighted portfolio
init_guess = np.ones(len(asset_columns)) / len(asset_columns)

# === Step 5: Perform the Optimization for Maximum Diversification Portfolio ===

result = minimize(
    fun=max_diversification_objective,
    x0=init_guess,
    args=(asset_std, cov_matrix),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Check if the optimization was successful
if not result.success:
    raise ValueError(f"Optimization failed: {result.message}")

# Extract the optimized weights for the maximum diversification portfolio
optimal_weights = result.x

# === Step 6: Calculate Portfolio Volatility and Diversification Ratio ===

portfolio_std = portfolio_volatility(optimal_weights, cov_matrix)
div_ratio = diversification_ratio(optimal_weights, asset_std, cov_matrix)

# === Step 7: Calculate Risk Contributions ===

risk_contribs = risk_contribution(optimal_weights, cov_matrix)

# Create a DataFrame to display weights, risk contributions, and risk contribution percentages
risk_contrib_table = pd.DataFrame({
    'Asset': asset_columns,
    'Weight %': optimal_weights * 100,
    'Risk Contribution': risk_contribs,
    'Risk Contribution %': risk_contribs / portfolio_std * 100  # Convert to percentage
})

# Convert the weights to percentages and round to two decimal places
risk_contrib_table['Weight %'] = risk_contrib_table['Weight %'].round(2)
risk_contrib_table['Risk Contribution %'] = risk_contrib_table['Risk Contribution %'].round(2)

# Display the Risk Contribution Table with updated weights and contributions
print(f"Portfolio Standard Deviation (monthly): {portfolio_std:.6f}")
print(f"Diversification Ratio: {div_ratio:.4f}\n")
print("Risk Contribution to Portfolio Standard Deviation:")
print(risk_contrib_table.to_string(index=False))

# === Step 8: Calculate Portfolio Returns ===

# Calculate portfolio returns using the optimized weights
portfolio_returns = assets.dot(optimal_weights)

# Extract benchmark returns
benchmark_returns = data[vasix_col].loc[portfolio_returns.index]

# Combine portfolio and benchmark returns into a single DataFrame
performance_df = pd.DataFrame({
    'Maximum Diversification Portfolio': portfolio_returns,
    'VASIX (Benchmark)': benchmark_returns
}).dropna()

# Ensure the index is datetime and sorted
performance_df.index = pd.to_datetime(performance_df.index)
performance_df = performance_df.sort_index()

# === Step 9: Define Performance Metrics Functions ===

def calculate_cagr(df, column_name):
    """Calculate the Compound Annual Growth Rate (CAGR)."""
    if df.index.nunique() < 2:
        raise ValueError(f"Date range is too short to calculate CAGR for {column_name}.")
    
    # Calculate the number of years between the first and last date
    days_diff = (df.index[-1] - df.index[0]).days
    if days_diff <= 0:
        raise ValueError("Date range is too short to calculate CAGR.")
    
    years = days_diff / 365.25
    cumulative_return = (1 + df[column_name]).prod()
    
    return (cumulative_return ** (1 / years) - 1) * 100  # Convert to percentage

def calculate_max_drawdown(df, column_name):
    """Calculate the Maximum Drawdown."""
    cumulative_return = (1 + df[column_name]).cumprod()
    rolling_max = cumulative_return.cummax()
    drawdown = (cumulative_return - rolling_max) / rolling_max
    return drawdown.min() * 100  # Convert to percentage

# === Step 10: Calculate Annual Returns ===

def calculate_annual_returns(df, column_name):
    """Calculate annual returns for the portfolio."""
    # Resample data to year-end using 'YE' for year-end
    annual_returns = df[column_name].resample('YE').apply(lambda x: (1 + x).prod() - 1)
    
    # Convert to percentage
    return (annual_returns * 100).round(2)

# Calculate annual returns for both portfolio and benchmark
annual_returns_portfolio = calculate_annual_returns(performance_df, 'Maximum Diversification Portfolio')
annual_returns_benchmark = calculate_annual_returns(performance_df, 'VASIX (Benchmark)')

# Combine annual returns into a DataFrame
annual_returns_table = pd.DataFrame({
    'Maximum Diversification Portfolio (%)': annual_returns_portfolio,
    'VASIX (Benchmark) (%)': annual_returns_benchmark
})

# Display the annual return table
print("\nAnnual Return Table:")
print(annual_returns_table.to_string())

# === Step 11: Calculate Performance Metrics ===

if performance_df.index.nunique() < 2:
    print("Date range is too short for CAGR calculation. Skipping CAGR.")
else:
    try:
        # Calculate CAGR for both portfolio and benchmark
        cagr_portfolio = calculate_cagr(performance_df, 'Maximum Diversification Portfolio')
        cagr_benchmark = calculate_cagr(performance_df, 'VASIX (Benchmark)')
        
        # Calculate Standard Deviation (annualized and converted to percentage)
        std_portfolio = portfolio_returns.std() * np.sqrt(12) * 100  # Assuming monthly returns
        std_benchmark = benchmark_returns.std() * np.sqrt(12) * 100
        
        # Calculate Maximum Drawdown for both portfolio and benchmark
        mdd_portfolio = calculate_max_drawdown(performance_df, 'Maximum Diversification Portfolio')
        mdd_benchmark = calculate_max_drawdown(performance_df, 'VASIX (Benchmark)')
        
        # Compile the performance metrics into a DataFrame
        performance_metrics = pd.DataFrame({
            'CAGR (%)': [cagr_portfolio, cagr_benchmark],
            'Standard Deviation (%)': [std_portfolio, std_benchmark],
            'Maximum Drawdown (%)': [mdd_portfolio, mdd_benchmark]
        }, index=['Maximum Diversification Portfolio', 'VASIX (Benchmark)'])

        # Format and display the Performance Metrics with two decimal places
        print("\nPerformance Metrics (in percentages):")
        print(performance_metrics.round(2).to_string())

    except ValueError as e:
        print(f"Error calculating performance metrics: {e}")


In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# === Helper Functions ===

def portfolio_volatility(weights, cov_matrix):
    """Calculate the portfolio volatility."""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def risk_contribution(weights, cov_matrix):
    """Calculate the risk contribution of each asset to the portfolio."""
    total_vol = portfolio_volatility(weights, cov_matrix)
    marginal_contrib = np.dot(cov_matrix, weights)
    return (marginal_contrib * weights) / total_vol

def calculate_cagr(df, column_name):
    """Calculate the Compound Annual Growth Rate (CAGR)."""
    days_diff = (df.index[-1] - df.index[0]).days
    years = days_diff / 365.25
    cumulative_return = (1 + df[column_name]).prod()
    return (cumulative_return ** (1 / years) - 1) * 100  # Convert to percentage

def calculate_max_drawdown(df, column_name):
    """Calculate the Maximum Drawdown."""
    cumulative_return = (1 + df[column_name]).cumprod()
    rolling_max = cumulative_return.cummax()
    drawdown = (cumulative_return - rolling_max) / rolling_max
    return drawdown.min() * 100  # Convert to percentage

def calculate_annual_returns(df, column_name):
    """Calculate annual returns for the portfolio."""
    annual_returns = df[column_name].resample('YE').apply(lambda x: (1 + x).prod() - 1)
    return (annual_returns * 100).round(2)  # Convert to percentage and round

# === Portfolio Optimizations ===

def optimize_risk_parity(cov_matrix):
    def risk_parity_objective(weights, cov_matrix):
        contribs = risk_contribution(weights, cov_matrix)
        return np.std(contribs)

    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1) for _ in range(len(cov_matrix))]
    init_guess = np.ones(len(cov_matrix)) / len(cov_matrix)

    result = minimize(risk_parity_objective, init_guess, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def optimize_min_variance(cov_matrix):
    def min_variance_objective(weights, cov_matrix):
        return portfolio_volatility(weights, cov_matrix)

    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1) for _ in range(len(cov_matrix))]
    init_guess = np.ones(len(cov_matrix)) / len(cov_matrix)

    result = minimize(min_variance_objective, init_guess, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def optimize_max_diversification(cov_matrix, asset_std):
    def diversification_ratio(weights, asset_std, cov_matrix):
        weighted_volatility_sum = np.dot(weights, asset_std)
        portfolio_vol = portfolio_volatility(weights, cov_matrix)
        return weighted_volatility_sum / portfolio_vol

    def max_diversification_objective(weights, asset_std, cov_matrix):
        return -diversification_ratio(weights, asset_std, cov_matrix)

    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1) for _ in range(len(cov_matrix))]
    init_guess = np.ones(len(cov_matrix)) / len(cov_matrix)

    result = minimize(max_diversification_objective, init_guess, args=(asset_std, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# === Load Data ===

# Define the file path
file_path = 'Opportunity_Set.xlsx'

# Read the Excel file, parse 'Date' as datetime, and set it as the index
data = pd.read_excel(file_path, parse_dates=['Date'], index_col='Date')

# Exclude the benchmark and keep only asset columns
asset_columns = [col for col in data.columns if col != 'Vanguard LifeStrategy Income Fund (VASIX)']
assets = data[asset_columns].apply(pd.to_numeric, errors='coerce').dropna()

# Calculate the covariance matrix and asset volatilities (std)
cov_matrix = assets.cov()
asset_std = assets.std()

# === Perform Optimizations for all Portfolios ===

weights_risk_parity = optimize_risk_parity(cov_matrix)
weights_min_variance = optimize_min_variance(cov_matrix)
weights_max_diversification = optimize_max_diversification(cov_matrix, asset_std)

# === Portfolio Returns ===

# Calculate returns using the optimized weights
returns_risk_parity = assets.dot(weights_risk_parity)
returns_min_variance = assets.dot(weights_min_variance)
returns_max_diversification = assets.dot(weights_max_diversification)

# Combine portfolio returns into a DataFrame and include VASIX benchmark returns
performance_df = pd.DataFrame({
    'Risk Parity': returns_risk_parity,
    'Minimum Variance': returns_min_variance,
    'Maximum Diversification': returns_max_diversification,
    'VASIX (Benchmark)': data['Vanguard LifeStrategy Income Fund (VASIX)']
})

# === Build Tables for Weights and Risk Contributions ===

def build_weight_table():
    """Create a table for weights only."""
    table = pd.DataFrame({
        'Asset': asset_columns,
        'Risk Parity': weights_risk_parity * 100,
        'Min Variance': weights_min_variance * 100,
        'Max Diversification': weights_max_diversification * 100
    }).round(2)
    return table

def build_risk_contrib_table():
    """Create a table for risk contributions only."""
    risk_contrib_rp = risk_contribution(weights_risk_parity, cov_matrix) / portfolio_volatility(weights_risk_parity, cov_matrix) * 100
    risk_contrib_mv = risk_contribution(weights_min_variance, cov_matrix) / portfolio_volatility(weights_min_variance, cov_matrix) * 100
    risk_contrib_md = risk_contribution(weights_max_diversification, cov_matrix) / portfolio_volatility(weights_max_diversification, cov_matrix) * 100
    
    table = pd.DataFrame({
        'Asset': asset_columns,
        'Risk Parity': risk_contrib_rp,
        'Min Variance': risk_contrib_mv,
        'Max Diversification': risk_contrib_md
    }).round(2)
    return table

# === Annual Returns Table ===

def build_annual_returns_table():
    """Create a table for annual returns."""
    annual_returns_rp = calculate_annual_returns(performance_df, 'Risk Parity')
    annual_returns_mv = calculate_annual_returns(performance_df, 'Minimum Variance')
    annual_returns_md = calculate_annual_returns(performance_df, 'Maximum Diversification')
    annual_returns_vasix = calculate_annual_returns(performance_df, 'VASIX (Benchmark)')
    
    table = pd.DataFrame({
        'Risk Parity (%)': annual_returns_rp,
        'Minimum Variance (%)': annual_returns_mv,
        'Maximum Diversification (%)': annual_returns_md,
        'VASIX (Benchmark) (%)': annual_returns_vasix
    })
    return table

# === Performance Metrics Table ===

def build_performance_metrics(df, name):
    """Calculate CAGR, Standard Deviation, and Maximum Drawdown."""
    cagr = calculate_cagr(df, name)
    std_dev = df[name].std() * np.sqrt(12) * 100  # Annualized standard deviation
    mdd = calculate_max_drawdown(df, name)
    
    return pd.Series({'CAGR (%)': cagr, 'Standard Deviation (%)': std_dev, 'Maximum Drawdown (%)': mdd}).round(2)

def build_performance_metrics_table():
    """Create a table for performance metrics."""
    performance_metrics = pd.DataFrame({
        'Risk Parity': build_performance_metrics(performance_df, 'Risk Parity'),
        'Minimum Variance': build_performance_metrics(performance_df, 'Minimum Variance'),
        'Maximum Diversification': build_performance_metrics(performance_df, 'Maximum Diversification'),
        'VASIX (Benchmark)': build_performance_metrics(performance_df, 'VASIX (Benchmark)')
    })
    return performance_metrics

# === Generate Tables ===

# Generate the tables
weights_table = build_weight_table()
risk_contrib_table = build_risk_contrib_table()
annual_returns_table = build_annual_returns_table()
performance_metrics_table = build_performance_metrics_table()

# Display the tables
print("Portfolio Weights Table (%):")
print(weights_table.to_string(index=False))

print("\nRisk Contribution Table (%):")
print(risk_contrib_table.to_string(index=False))

print("\nAnnual Return Table (%):")
print(annual_returns_table.to_string())

print("\nPerformance Metrics Table:")
print(performance_metrics_table.round(2).to_string())

Portfolio Weights Table (%):
                                           Asset  Risk Parity  Min Variance  Max Diversification
             Vanguard Total World Stock ETF (VT)        11.62         15.84                23.15
    PIMCO 25+ Year Zero Coupon US Trs ETF (ZROZ)         4.75          2.01                 3.38
             AQR Diversified Arbitrage I (ADAIX)        15.04         22.46                 8.49
                        iShares Gold Trust (IAU)         4.05          3.29                 0.00
                 Bitcoin Market Price USD (^BTC)         0.92          0.00                 0.46
AQR Risk-Balanced Commodities Strategy I (ARCIX)         4.27          0.31                 4.08
                 AQR Long-Short Equity I (QLEIX)         4.58          0.00                 0.00
          AQR Style Premia Alternative I (QSPIX)         3.82          0.00                 0.00
             AQR Equity Market Neutral I (QMNIX)         5.59          9.09                 9.16
 